<a href="https://colab.research.google.com/github/HDWilliams/mnistDigitClassifier/blob/master/mnistDigitClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#import necessary libraries
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, datasets


In [0]:
#set up data transforms and import data
transform = transforms.Compose([transforms.ToTensor()])

#set up train and test dataloaders
trainset = datasets.MNIST('../mnist_data', download=True, train=True, transform=transform)
train_dataloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True)

testset = datasets.MNIST('../mnist_data', download=True, train=False, transform=transform)
test_dataloader = torch.utils.data.DataLoader(testset, batch_size=16, shuffle = True)


In [0]:
#visualize the images
#TODO: Fix display issues
def show_img(num):
  w=10
  h=10
  fig=plt.figure(figsize=(num, 1))
  columns = 28
  rows = 28
  for image, label in train_dataloader:
    for i in range(1, columns*rows +1):
      fig.add_subplot(rows, columns, i)
      plt.imshow(image.view(-1, 28))
    break
  plt.show()
show_img(4)

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
#set up a Sequential model input 28*28, ouput 10 catagories
import torch.nn.functional as F
import torch.nn as nn

model = nn.Sequential(nn.Linear(28*28, 128), 
                      nn.ReLU(), 
                      nn.Linear(128, 64), 
                      nn.ReLU(), 
                      nn.Linear(64, 10), 
                      nn.LogSoftmax(dim=1)).to(device)

#loss criterion
criterion = nn.NLLLoss()
#optimizer
lr=.01
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


In [17]:
#set up train test code
epochs = 10

for i in range(epochs):
  running_loss = 0
  for image, label in train_dataloader:
    optimizer.zero_grad()
    image = image.view(image.shape[0], -1)
    
    image, label = image.to(device), label.to(device)
    output = model(image)
    loss = criterion(output, label)
    loss.backward()
    optimizer.step()
    
    running_loss += loss.item()
    print(running_loss)
    

0.29895344376564026
0.35020340979099274
0.5962032526731491
0.6039091199636459
0.744473472237587
0.7924400866031647
1.2368015348911285
1.3128375560045242
1.432335078716278
1.74812912940979
1.8118179440498352
1.931909054517746
2.641026884317398
2.7170319706201553
3.478283479809761
3.656584307551384
4.175634369254112
5.043288096785545
5.129010319709778
5.9500919580459595
6.148510247468948
6.342780873179436
7.013097509741783
7.162851691246033
7.5764608681201935
7.952907353639603
7.992145195603371
8.15393690764904
8.55811332166195
9.260440573096275
9.44052092730999
9.642382562160492
9.755480095744133
9.935389399528503
10.118049889802933
10.160234466195107
10.185463771224022
10.438285931944847
10.556387305259705
10.684291809797287
10.93050953745842
11.043208062648773
11.114763423800468
11.325679898262024
11.452193483710289
12.22230564057827
12.429455265402794
12.638180255889893
13.43527638912201
13.694067180156708
15.215332806110382
15.356995776295662
15.438424199819565
15.679558128118515
15

KeyboardInterrupt: ignored